# YamboConvergence: automated GW/BSE convergence

This is the highest-level workflow of the plugin. It runs an automated GW or BSE convergence.
Simulations are organized on the fly without human intervention.

The idea is to create a builder instance, which collects all the inputs needed for the simulation, then submit it by means of the aiida "submit" function.

In [1]:
%aiida  
## this is a magic line function, it can be substitued by: ---

from aiida import orm

## 1 - Creation of YamboConvergence instance using protocols

In this tutorial the creation of the builder, for a given workflow, is provided by means of the get_builder_from_protocol function.
This is, for now, just a way to automatically populated all the needed inputs, allowing the user to partially skip the long part of input definition as done in examples like the one contained in ``aiida_yambo/examples_hBN/workflows/yambo_convergence.py``.


It is not meant to be an already tested set of default values as instead is done for the aiida-quantumespresso plugin. 

In [2]:
from aiida.plugins import WorkflowFactory

In [3]:
YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')

## minimal inputs needed for protocols

In [4]:
options = {
    'pwcode_id': 'pw-6.8@hydralogin', 
    'pseudo_family':"PseudoDojo/0.4/LDA/SR/standard/upf",
    'yamboprecode_id':'p2y-devel@hydralogin',
    'yambocode_id':'yambo-RIMW@hydralogin',
    'protocol':'fast',
    #'parent_id':86354, #take your previously nscf id (pk)
    'structure_id':77395,
}

In [5]:
#YamboWorkflow.get_builder_from_protocol??

In [6]:
from aiida_quantumespresso.common.types import ElectronicType

In [7]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= load_node(options['structure_id']),
            overrides={},
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='gw', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXp = 150
GbndRnge = 150
NGsBlkXp = 2 Ry
FFTGvecs = 9 Ry


kpoint mesh for nscf: [6, 6, 6]


/scratch/mbonacci/vaiida/lib/python3.8/site-packages/aiida_quantumespresso/calculations/pw.py:204: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by setting the validator of the `PwCalculation` port to `PwCalculation.validate_inputs_base`.
  warnings.warn(


In [8]:
#You can also try different protocols:
    
YamboConvergence.get_available_protocols()

{'fast': {'description': 'Fast protocol for a GW convergence: grid -> poor; thresholds -> poor'},
 'moderate': {'description': 'Moderate protocol for a GW convergence: grid -> enough good for standard materials; thresholds -> moderate (5 percent)'},
 'precise': {'description': 'precise protocol for a GW convergence: grid -> same as moderate; thresholds -> precise (1 percent)'},
 'molecule': {'description': 'Moderate protocol for a GW convergence in molecules'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [9]:
builder.ywfl.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 30.0,
  'ecutrho': 240.0,
  'force_symmorphic': True,
  'nbnd': 150},
 'ELECTRONS': {'electron_maxstep': 80, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

In [10]:
builder.ywfl.yres.yambo.parameters.get_dict()

{'arguments': ['dipoles', 'ppa', 'HF_and_locXC', 'gw0'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'NGsBlkXp': [2, 'Ry'],
  'FFTGvecs': [9, 'Ry'],
  'BndsRnXp': [[1, 150], ''],
  'GbndRnge': [[1, 150], ''],
  'QPkrange': [[[1, 1, 32, 32]], '']}}

## 2 - Inputs completion

We have to include also the resources:

In [11]:
builder.ywfl.scf.pw.metadata.options = {
    'max_wallclock_seconds': 2*60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

## 3 - Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [12]:
overrides_scf = {
        'pseudo_family': 'PseudoDojo/0.4/LDA/SR/standard/upf', 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{'ecutwfc': 50.0,
                          'ecutrho': 200.0,},
                'ELECTRONS':{},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}
overrides_nscf = {
        'pseudo_family': 'PseudoDojo/0.4/LDA/SR/standard/upf', 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{'ecutwfc': 50.0,
                          'ecutrho': 200.0,},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "NGsBlkXp": [4, "Ry"],
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 3*60*60, # in seconds
                    'resources': {
                            "num_machines": 2, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

#Be careful with the mesh choice!!! 
overrides_meta = {
        'FFTGvecs': {
            'start_ratio': 0.25,
            'stop_ratio': 0.7,
            'delta_ratio': 0.1,
            'max_ratio': 1,
        },
        'bands': {
            'start': 50,
            'stop': 400,
            'delta': 50,
            'max': 600,
            'ratio':[10,25,50],
        },
        'G_vectors': {
            'start': 2,
            'stop': 8,
            'delta': 1,
            'max': 10,
        },
        'kpoint_density': {
            'start': 0.8,
            'stop': 0.2,
            'delta': 3,
            'max': 0.1,
        } ,
        'conv_thr_k': 10,
        'conv_thr_bG': 10,
        'conv_thr_FFT': 10,
        'conv_thr_units': '%', # 'eV'

        
    }

        
overrides_wfl_settings = {
        
        'what':['gap_'],
        'bands_nscf_update':'full-step',
        'skip_pre':False,   
        'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
        
    }

overrides = {
        'meta_parameters':overrides_meta,
        'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
        'workflow_settings':overrides_wfl_settings,
    }


In [13]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='gw', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXp = 150
GbndRnge = 150
NGsBlkXp = 4 Ry
FFTGvecs = 15 Ry


kpoint mesh for nscf: [6, 6, 6]


In [31]:
builder.group_label = Str('Silicon/try') # verdi group create Silicon/try; all calculationsc are added to the group

In [15]:
builder.ywfl.scf.pw.metadata

{'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 16, 'num_cores_per_mpiproc': 1}, 'max_wallclock_seconds': 3600, 'withmpi': True, 'prepend_text': 'export OMP_NUM_THREADS=1', 'queue_name': 's3par'}}

## 3 RUN

In [16]:
from aiida.engine import submit

In [17]:
run = None

In [18]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

uuid: 79e3255b-aaf9-46c3-86da-0b10b8548554 (pk: 89059) (aiida.workflows:yambo.yambo.yamboconvergence)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [23]:
run.is_finished_ok

True

In [24]:
run.outputs. #+TAB

SyntaxError: invalid syntax (708973465.py, line 1)

The converged parameters can be obtained via the "infos" output Dict:

In [25]:
run.outputs.infos.get_dict()

{'gap_': 1.026851127116,
 'E_ref': 1.1462764156663,
 'BndsRnXp': 110.0,
 'FFTGvecs': 12,
 'GbndRnge': 110.0,
 'NGsBlkXp': 5.0,
 'kpoint_mesh': [7, 7, 7],
 'extrapolation': 1.1548236376551}

The full convergence history can be visualized in a table form using pandas:

In [26]:
import pandas as pd

In [27]:
history = run.outputs.history.get_dict()

In [28]:
history_table = pd.DataFrame(history)

In [29]:
history_table

,gap_,uuid,failed,useful,BndsRnXp,FFTGvecs,GbndRnge,NGsBlkXp,global_step,kpoint_mesh,parameters_studied
0,1.106919,7f687803-8958-4807-857a-eabc39a2f27f,False,False,150,12,150,4.0,1,"[4, 4, 4]",FFTGvecs
1,1.109694,084b6d90-7aa3-4a4b-942f-23219693cddf,False,False,150,22,150,4.0,2,"[4, 4, 4]",FFTGvecs
2,1.109486,e61184fe-fa0c-417d-aeba-8cc8156e8081,False,False,150,27,150,4.0,3,"[4, 4, 4]",FFTGvecs
3,1.109428,84443b47-b5a3-471e-8fc7-8a100ad399b7,False,False,150,35,150,4.0,4,"[4, 4, 4]",FFTGvecs
4,1.106919,7f687803-8958-4807-857a-eabc39a2f27f,False,False,150,12,150,4.0,5,"[4, 4, 4]",kpoint_mesh
5,0.960021,ce33e842-c988-4640-b20a-e86dacfb25ef,False,False,150,12,150,4.0,6,"[7, 7, 7]",kpoint_mesh
6,0.971251,884078cf-9d80-4301-8902-0f62ad5b9091,False,False,150,12,150,4.0,7,"[10, 10, 10]",kpoint_mesh
7,0.963438,cf9d793a-eee3-4253-b486-799fdcf175e2,False,False,150,12,150,4.0,8,"[12, 12, 12]",kpoint_mesh
8,0.835478,a415b238-056d-4b07-9d24-8dae00f09510,False,False,60,12,60,2.0,9,"[7, 7, 7]","BndsRnXp, GbndRnge, NGsBlkXp"
9,0.841587,31ad3b67-ef3e-43ce-9a2a-e03582d62406,False,False,400,12,400,2.0,10,"[7, 7, 7]","BndsRnXp, GbndRnge, NGsBlkXp"


The last calculations can be obtained using:

In [30]:
history_table[history_table['useful']==True]

,gap_,uuid,failed,useful,BndsRnXp,FFTGvecs,GbndRnge,NGsBlkXp,global_step,kpoint_mesh,parameters_studied
20,1.026851,24a4bd74-54f5-4758-b61c-edf3da3e5c91,False,True,110,12,110,5.0,21,"[7, 7, 7]","BndsRnXp, GbndRnge, NGsBlkXp"
